In [1]:
from Stitcher import stitch_neuron
import datajoint as dj
import numpy as np
import datajoint as dj
import trimesh
import time

In [2]:
# testing which table to pull from
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


# Check the datajoint parent will be correct

In [3]:
#gets the possible segment ids
pinky.Mesh() & (pinky.AllenSoma() & "cell_class='excitatory'")

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
3,648518346341371119,669574,1316372,=BLOB=,=BLOB=
3,648518346349386137,85806,166966,=BLOB=,=BLOB=
3,648518346349470171,1254494,2496165,=BLOB=,=BLOB=
3,648518346349471156,865120,1721566,=BLOB=,=BLOB=
3,648518346349471500,803277,1599182,=BLOB=,=BLOB=
3,648518346349471562,1343736,2654580,=BLOB=,=BLOB=
3,648518346349471565,2232110,4423873,=BLOB=,=BLOB=


In [4]:
@schema
class ExcitatoryStitchedMesh(dj.Computed):
    definition="""
    -> pinky.Mesh
    ---
    n_vertices           : bigint           # number of vertices in this mesh
    n_triangles          : bigint           # number of triangles in this mesh
    vertices             : longblob         # x,y,z coordinates of vertices
    triangles            : longblob         # triangles (triplets of vertices)
    n_pieces             : int              # number of unconnected mesh pieces outside the largest mesh piece
    largest_piece_perc   : decimal(6,5)     # number of faces percentage of largest mesh piece in respect to total mesh
    outside_perc         : decimal(6,5)     # number of faces percentage of mesh outside the biggest mesh piece
    n_stitched           : int              # number of mesh pieces stitched back to main mesh
    stitched_addon_perc  : decimal(6,5)     # number of faces percentage of pieces that were stitched back in respect to largest mesh piece
    n_unstitched         : int              # number of mesh pieces remaining unstitched back to main mesh        
    unstitched_perc     : decimal(6,5)     # number of faces percentage of pieces that were not in respect to largest mesh piece
    
    
    """
    
    key_source = pinky.Mesh() & ((pinky.AllenSoma() & "cell_class='excitatory'").proj())
    
    
    def make(self, key):
        
        segment_id = key["segment_id"]
        key = dict(segment_id=segment_id,segmentation=3)
        vertices,triangles = (pinky.Mesh & key).fetch("vertices","triangles")

        verts = vertices[0]
        faces = triangles[0]

        print(f"\n\n\n*****Running Neuron Stitcher for {segment_id}**********")

        [n_vertices,
         n_triangles,
         vertices,
         triangles,
         n_pieces,
         largest_piece_perc,
         outside_perc,
         n_stitched,
         stitched_addon_perc,
         n_unstitched,
         unstitched_perc] = stitch_neuron(segment_id=segment_id,
                                                      vertices=verts,
                                                      faces=faces,
                                                    import_from_off=False,
                                                     load_meshes_flag = False,
                                                 save_meshes_flag = True,
                                                    pymeshfix_flag = False
                                                   )
        
        #insert dummy dictionary into correspondence table
        insert_key = dict(key,
                          n_vertices=n_vertices,
                         n_triangles=n_triangles,
                         vertices=vertices,
                         triangles=triangles,
                         n_pieces=n_pieces,
                         largest_piece_perc=largest_piece_perc,
                         outside_perc=outside_perc,
                         n_stitched=n_stitched,
                         stitched_addon_perc=stitched_addon_perc,
                         n_unstitched=n_unstitched,
                         unstitched_perc=unstitched_perc)
                        
        
        self.insert1(insert_key,skip_duplicates=True)
        
        
                         

In [ ]:
start = time.time()
ExcitatoryStitchedMesh.populate(reserve_jobs=True)
print(time.time() - start)




*****Running Neuron Stitcher for 648518346349504754**********
generating child and main meshes
------Starting the stitcher vp2-------
There were 4832 pieces after mesh split
There were 573 pieces found after size threshold
max_index = 3
max_face_len = 3172922
Total time for Mesh Cleansing: 1551.297593832016
Number of outside child meshes =  79
Main mesh and outside child meshes make up 0.9347609077040987% of original mesh
Saved child and main meshes at stitch_mesh_saved/648518346349504754_30_main_and_child_meshes_array.npz
 face length 3172922 using optimized facets with 3 neighbors
Filtering edges by size finished: 57.54092502593994, facet # = 170348
Filtering by convexity and getting centers took: 69.3353340625763, facet # = 31969
Finished facets for main mesh: 132.84698033332825 with facet length = 31969
Finished 31969 facets for main mesh: 132.84730768203735
Filtering edges by size finished: 0.4839344024658203, facet # = 2259
Filtering by convexity and getting centers took: 0.90

In [ ]:
#(schema.jobs & "table_name='__excitatory_stitched_mesh'").delete()
#schema.jobs

In [ ]:
"""
Things didn't think about: 
1) if there are no child meshes to be stitched 
2) If there is only one mesh


"""